# PythonOperator

De multe ori în cadrul unui pipeline trebuie să executăm un anumit cod de Python. Pentru a executa cod de Python, putem să ne folosim de operatorul PythonOperator prin care se execută o funcție de Python

Pentru proiectul respectiv o să ne descărcăm ceva date pe laptop utilizând code de Python

Metoda pe care o să o utilizăm este următoarea:


In [ ]:
def download_rates():
    BASE_URL = "https://gist.githubusercontent.com/marclamberti/f45f872dea4dfd3eaa015a4a1af4b39b/raw/"
    ENDPOINTS = {
        'USD': 'api_forex_exchange_usd.json',
        'EUR': 'api_forex_exchange_eur.json'
    }
    with open('/opt/airflow/dags/files/forex_currencies.csv') as forex_currencies:
        reader = csv.DictReader(forex_currencies, delimiter=';')
        for idx, row in enumerate(reader):
            base = row['base']
            with_pairs = row['with_pairs'].split(' ')
            indata = requests.get(f"{BASE_URL}{ENDPOINTS[base]}").json()
            outdata = {'base': base, 'rates': {}, 'last_update': indata['date']}
            for pair in with_pairs:
                outdata['rates'][pair] = indata['rates'][pair]
            with open('/opt/airflow/dags/files/forex_rates.json', 'a') as outfile:
                json.dump(outdata, outfile)
                outfile.write('\n')

Metoda de mai sus folosește atât librăria de requests, cât și cea de json. Pe lângă aceste librării trebuie să importăm și acest operator de Python. Codul complet (împreună cu operatorul de Python implementat) o să arate așa:

In [ ]:
import csv
import requests
import json

from airflow.operators.python import PythonOperator

def download_rates():
    BASE_URL = "https://gist.githubusercontent.com/marclamberti/f45f872dea4dfd3eaa015a4a1af4b39b/raw/"
    ENDPOINTS = {
        'USD': 'api_forex_exchange_usd.json',
        'EUR': 'api_forex_exchange_eur.json'
    }
    with open('/opt/airflow/dags/files/forex_currencies.csv') as forex_currencies:
        reader = csv.DictReader(forex_currencies, delimiter=';')
        for idx, row in enumerate(reader):
            base = row['base']
            with_pairs = row['with_pairs'].split(' ')
            indata = requests.get(f"{BASE_URL}{ENDPOINTS[base]}").json()
            outdata = {'base': base, 'rates': {}, 'last_update': indata['date']}
            for pair in with_pairs:
                outdata['rates'][pair] = indata['rates'][pair]
            with open('/opt/airflow/dags/files/forex_rates.json', 'a') as outfile:
                json.dump(outdata, outfile)
                outfile.write('\n')


download_forex_rates = PythonOperator(
    task_id="download_forex_rates",
    python_callable=download_rates
)

Tot codul de Python pentru DAG în airlfow o să arate acuma așa:

In [ ]:
from airflow import DAG
from airflow.providers.http.sensors.http import HttpSensor
from airflow.sensors.filesystem import FileSensor
from airflow.operators.python import PythonOperator

import json
import csv
import requests
from datetime import datetime, timedelta


# Define the default_args
default_args = {
    "owner": "airflow",
    "retries": 3,
    "retry_delay": timedelta(minutes=5),
    "email_on_failure": False,
    "email": "georgiuandrei05@gmail.com"
}

# Define the function that donwloads the data from the API
def download_rates():
    BASE_URL = "https://gist.githubusercontent.com/marclamberti/f45f872dea4dfd3eaa015a4a1af4b39b/raw/"
    ENDPOINTS = {
        'USD': 'api_forex_exchange_usd.json',
        'EUR': 'api_forex_exchange_eur.json'
    }
    with open('/opt/airflow/dags/files/forex_currencies.csv') as forex_currencies:
        reader = csv.DictReader(forex_currencies, delimiter=';')
        for idx, row in enumerate(reader):
            base = row['base']
            with_pairs = row['with_pairs'].split(' ')
            indata = requests.get(f"{BASE_URL}{ENDPOINTS[base]}").json()
            outdata = {'base': base, 'rates': {}, 'last_update': indata['date']}
            for pair in with_pairs:
                outdata['rates'][pair] = indata['rates'][pair]
            with open('/opt/airflow/dags/files/forex_rates.json', 'a') as outfile:
                json.dump(outdata, outfile)
                outfile.write('\n')

# Define the DAG
with DAG(
    dag_id = 'test_forex_data_pipeline',
    description='Testing the Data Forex Pipeline from Udemy Course',
    schedule_interval="@daily",
    start_date=datetime(2024, 1, 1),
    catchup=False
    ) as dag:
    
    # Define the HttpSensor task
    is_forex_rates_available = HttpSensor(
        task_id="is_forex_rates_available",
        http_conn_id="forex_api",
        endpoint="marclamberti/f45f872dea4dfd3eaa015a4a1af4b39b",
        response_check=lambda response: "rates" in response.text,
        poke_interval=5,
        timeout=25
    )

    # Define the FileSensor task
    is_forex_file_available = FileSensor(
        task_id="is_forex_file_available",
        fs_conn_id="forex_path",
        filepath="forex_currencies.csv",
        poke_interval=5,
        timeout=20
    )

    # Define the PythonOperator task
    download_forex_rates = PythonOperator(
        task_id="download_forex_rates",
        python_callable=download_rates
    )



Acesta este modul prin care putem să executăm cod de Python în cadrul Airflow.